# Chemoinformatics using Python: Predict Solubility with ML and DL models

## Part III:  DL using Pytorch 

### Import Packages

In [1]:
# rdkit package for data formatting
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem, Descriptors,rdMolDescriptors, Draw, PandasTools
from rdkit.Chem.Draw import IPythonConsole
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.rdMolDescriptors import CalcExactMolWt
from rdkit.Chem.Descriptors import MolLogP
from rdkit.Avalon import pyAvalonTools

#general package for data science
import pandas as pd
import numpy as np
import random
import seaborn as sn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

#torch package for Deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch import optim

from tqdm import tqdm

import session_info
session_info.show()

# graphical card for pytorch
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### List of definitions

In [2]:
def uniq_smiles (smiles):
    mols =[Chem.MolFromSmiles(smil) for smil in smiles]
    smiles = [Chem.MolToSmiles(mol) for mol in mols]
    return smiles

def gen_avfpts_2(df):
    Av_fpts = [pyAvalonTools.GetAvalonFP(struct, nBits = 1600) for struct in tqdm (df.Structure)]
    return np.array(Av_fpts)

def generate_descriptors(df,verbose = False):
    mol_liste= [Chem.MolFromSmiles(element) for element in df.smiles]
    base_data= np.arange(1,1)
    i=0
    for mol in mol_liste:
        descr_Mw= CalcExactMolWt(mol)
        descr_logP= MolLogP(mol)
        
        row= np.array([descr_Mw,descr_logP,
                      ])
        if (i==0):
            base_data= row
        else:
            base_data = np.vstack([base_data,row])
        i=i+1
        
    colnames= ["MolWt", "logP"]
    descriptor= pd.DataFrame(data= base_data, columns= colnames)
    
    return descriptor

def generate_dataframe(path):
    df=pd.read_csv(path)
    Uniq_Smiles = uniq_smiles(df.smiles)
    df['smiles'] = Uniq_Smiles
    duplicate_smile= df[df['smiles'].duplicated()]['smiles'].values
    df[df['smiles'].isin(duplicate_smile)].sort_values(by= ['smiles'])
    df_clean= df.drop_duplicates(subset=['smiles']).reset_index(drop= True)
    PandasTools.AddMoleculeColumnToFrame(df_clean, 'smiles', 'Structure')
    av_fpts=gen_avfpts_2(df_clean)
    dataframe_fpts= pd.DataFrame(av_fpts)
    df_descrip= generate_descriptors(df_clean)
    dataset= dataframe_fpts.join(df_descrip, how='left').join(df_clean['logS'], how='left')
    
    return dataset

#### Import of the dataset and generate features for training and test

In [3]:
path=r"C:\Users\sylv_\Desktop\GitHub\SolCuration\clean\aqsol_stand.csv"
path_test=r"C:\Users\sylv_\Desktop\GitHub\SolCuration\clean\phys_stand.csv"

dataset_aqsol=generate_dataframe(path)
dataset_phys=generate_dataframe(path_test)

print(dataset_aqsol.head())
dataset_phys.head()

100%|██████████| 2001/2001 [00:00<00:00, 3535.33it/s]


   0  1  2  3  4  5  6  7  8  9  ...  1593  1594  1595  1596  1597  1598  \
0  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
1  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
2  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
3  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
4  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   

   1599       MolWt    logP    logS  
0     0   56.037222 -1.3084 -4.7424  
1     0  183.852324  2.2474 -1.7415  
2     0  183.852324  2.2474 -1.3195  
3     0  327.673348  3.1773 -3.1404  
4     0  249.762836  2.4547 -1.9113  

[5 rows x 1603 columns]


,0,1,2,3,4,5,6,7,8,9,...,1593,1594,1595,1596,1597,1598,1599,MolWt,logP,logS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,327.673348,3.1773,-3.140
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,249.762836,2.4547,-1.808
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,341.688998,3.2182,-2.732
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,162.004412,2.7140,-2.294
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,171.852324,1.7337,-1.165


#### Normalization of the data

In [4]:
# Concatenate the two DataFrames vertically with their columns to normalize
combined_df = pd.concat([dataset_aqsol.iloc[:, -3:-1], dataset_phys.iloc[:, -3:-1]], axis=0)

# Apply the scaler to the data
scaler = StandardScaler()
normalized_data=scaler.fit_transform(combined_df)

# Split the normalized data back into two separate DataFrames
dataset_aqsol_normalized = pd.DataFrame(normalized_data[:len(dataset_aqsol.iloc[:, -3:-1])], columns=dataset_aqsol.iloc[:, -3:-1].columns)
dataset_phys_normalized = pd.DataFrame(normalized_data[len(dataset_aqsol.iloc[:, -3:-1]):], columns=dataset_phys.iloc[:, -3:-1].columns)

print(dataset_aqsol_normalized.head())
dataset_phys_normalized

      MolWt      logP
0 -1.474927 -1.592555
1 -0.432384 -0.058573
2 -0.432384 -0.058573
3  0.740714  0.342588
4  0.105225  0.030857


,MolWt,logP
0,0.740714,0.342588
1,0.105225,0.030857
2,0.855035,0.360233
3,-0.610590,0.142719
4,-0.530264,-0.280185
...,...,...
1996,-1.246817,-0.273972
1997,-0.463245,0.172486
1998,-0.854970,-0.847005
1999,-0.609682,-0.210082


#### Replace the columns in the corresponding dataframe

In [5]:
dataset_aqsol.iloc[:, -3:-1] = dataset_aqsol_normalized
dataset_phys.iloc[:, -3:-1] = dataset_phys_normalized

In [6]:
dataset_aqsol.head()

,0,1,2,3,4,5,6,7,8,9,...,1593,1594,1595,1596,1597,1598,1599,MolWt,logP,logS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-1.474927,-1.592555,-4.7424
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.432384,-0.058573,-1.7415
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.432384,-0.058573,-1.3195
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.740714,0.342588,-3.1404
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.105225,0.030857,-1.9113


#### Converting  data of the training dataset into tensor

In [7]:
X = dataset_aqsol.iloc[:, :-1].values
y = dataset_aqsol.iloc[:, -1].values
X = torch.FloatTensor(X)
y = torch.FloatTensor(y)
y = y.view((-1, 1))

In [8]:
# take the size of the features and  
inputs_shape= X.shape[1]
print(inputs_shape)
print(X)

1602
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -1.4749, -1.5926],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.4324, -0.0586],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.4324, -0.0586],
        ...,
        [ 0.0000,  1.0000,  0.0000,  ...,  0.0000, -1.3689, -1.1124],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.9528, -0.8759],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.9447, -1.1369]])


#### Define the network architecture

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_features=inputs_shape, out_features=1024)
        self.fc1p = nn.LeakyReLU(0.1)
        self.fc0 = nn.Dropout(p=0.1)
        self.fc2 = nn.Linear(in_features=1024, out_features=512)
        self.fc2p = nn.LeakyReLU(0.1)
        self.fc3 = nn.Dropout(p=0.14)
        self.fc4 = nn.Linear(in_features=512, out_features=256)
        self.fc4p = nn.LeakyReLU(0.1)
        self.fc5 = nn.Dropout(p=0.18)
        self.fc6 = nn.Linear(in_features=256, out_features=128)
        self.fc6p = nn.LeakyReLU(0.1)
        self.fc7 = nn.Dropout(p=0.2)
        self.fc8 = nn.Linear(in_features=128, out_features=64)
        self.fc8p = nn.LeakyReLU(0.1)
        self.fc10 = nn.Linear(in_features=64, out_features=1)


    def forward(self, x):
        x = self.fc1(x)
        x = self.fc1p(x)
        x = self.fc0(x)
        x = self.fc2(x)
        x = self.fc2p(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc4p(x)
        x = self.fc5(x)
        x = self.fc6(x)
        x = self.fc6p(x)
        x = self.fc7(x)
        x = self.fc8(x)
        x = self.fc8p(x)
        x = self.fc10(x)

        return x

#### Construction of the training and validation dataset and their corresponding loaders

In [10]:
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    


dataset = MyDataset(X, y)
train_dataset, val_dataset = random_split(dataset, [round(len(dataset)*0.8),round(len(dataset)*0.2)])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

#### Instantiation of the network and definition of the loss, optimizer and some hyperparameters

In [11]:
net = Net().to(device) # important to direct the network on the gpu
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=4, verbose=True)
num_epoch=15

#### Training

In [12]:
#Training loop:
for epoch in range(num_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        inputs = inputs.to(device)
        labels= labels.to(device) # important to direct the values on the gpu for calculation

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    # Validation to see if the model is overfitting
    val_loss = 0.0
    with torch.no_grad():
        y_pred_list=[]
        y_true_list=[]
        for i, data in enumerate(val_loader, 0):
            inputs, labels = data
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            y_pred = outputs.detach().cpu().numpy().flatten()
            y_pred_list.extend(y_pred.tolist())
            y_true_list.extend(labels.tolist())
            r2_val = r2_score(y_true_list, y_pred_list)
            
    # update the scheduler for the learning rate
    scheduler.step(loss)
    
    #print some information on the training process
    print (f'epoch {epoch+1}/{num_epoch}, loss = {loss:.3f}')
    print('Epoch %d Validation loss: %.3f' % (epoch + 1, val_loss / len(val_loader))+f', r2_val = {r2_val:.3f}')

epoch 1/15, loss = 0.847
Epoch 1 Validation loss: 1.717, r2_val = 0.680
epoch 2/15, loss = 1.226
Epoch 2 Validation loss: 1.218, r2_val = 0.774
epoch 3/15, loss = 0.583
Epoch 3 Validation loss: 1.133, r2_val = 0.789
epoch 4/15, loss = 0.515
Epoch 4 Validation loss: 1.131, r2_val = 0.789
epoch 5/15, loss = 0.748
Epoch 5 Validation loss: 1.191, r2_val = 0.778
epoch 6/15, loss = 1.345
Epoch 6 Validation loss: 1.414, r2_val = 0.738
epoch 7/15, loss = 0.753
Epoch 7 Validation loss: 0.942, r2_val = 0.825
epoch 8/15, loss = 1.806
Epoch 8 Validation loss: 1.828, r2_val = 0.661
Epoch 00009: reducing learning rate of group 0 to 1.0000e-03.
epoch 9/15, loss = 1.217
Epoch 9 Validation loss: 0.921, r2_val = 0.830
epoch 10/15, loss = 0.828
Epoch 10 Validation loss: 0.814, r2_val = 0.849
epoch 11/15, loss = 0.678
Epoch 11 Validation loss: 0.810, r2_val = 0.849
epoch 12/15, loss = 0.628
Epoch 12 Validation loss: 0.796, r2_val = 0.852
epoch 13/15, loss = 0.662
Epoch 13 Validation loss: 0.799, r2_val = 

#### Verification of the network on one sample

In [13]:
with torch.no_grad():
    y_pred_ex= net(X[6])
    print(y_pred_ex)
    print(y[6])

tensor([-7.7632])
tensor([-6.9904])


#### Evaluating the model on the train set

In [14]:
net.eval()

Net(
  (fc1): Linear(in_features=1602, out_features=1024, bias=True)
  (fc1p): LeakyReLU(negative_slope=0.1)
  (fc0): Dropout(p=0.1, inplace=False)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc2p): LeakyReLU(negative_slope=0.1)
  (fc3): Dropout(p=0.14, inplace=False)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc4p): LeakyReLU(negative_slope=0.1)
  (fc5): Dropout(p=0.18, inplace=False)
  (fc6): Linear(in_features=256, out_features=128, bias=True)
  (fc6p): LeakyReLU(negative_slope=0.1)
  (fc7): Dropout(p=0.2, inplace=False)
  (fc8): Linear(in_features=128, out_features=64, bias=True)
  (fc8p): LeakyReLU(negative_slope=0.1)
  (fc10): Linear(in_features=64, out_features=1, bias=True)
)

In [15]:
# loader for the training dataset
eval_loader = DataLoader(dataset, batch_size=32, shuffle=False)

#forward pass 
with torch.no_grad():
    y_pred_list=[]
    y_true_list=[]
    for i, (inputs, labels) in enumerate(eval_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs = inputs.to(device)
            y_pred = net(inputs)
            y_pred = y_pred.detach().cpu().numpy().flatten()  # Convert predictions to a 1D numpy array

            y_pred_list.extend(y_pred.tolist())
            y_true_list.extend(labels.tolist())

In [16]:
# Obtain the r2 score on the training data
r2 = r2_score(y_true_list, y_pred_list)
print(f'Training set have a r2 of {r2:.5f}')

Training set have a r2 of 0.92404


#### Evaluating the model  on the test_dataset

In [17]:
# Transform the test dataset into tensors
X_test = dataset_phys.iloc[:, :-1].values
y_test = dataset_phys.iloc[:, -1].values
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)
y_test = y_test.view((-1, 1))

# torch dataset and loader for the test
dataset_test = MyDataset(X_test, y_test)

Test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False)

#forward pass to obtain the predict values
with torch.no_grad():
    y_pred_list=[]
    y_true_list=[]
    for i, (inputs, labels) in enumerate(Test_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs = inputs.to(device)
            y_pred = net(inputs)
            y_pred = y_pred.detach().cpu().numpy().flatten()  # Convert predictions to a 1D numpy array

            y_pred_list.extend(y_pred.tolist())
            y_true_list.extend(labels.tolist())
            
#Obtain the r2for the test dataset
r2 = r2_score(y_true_list, y_pred_list)
print(f'Test set have a r2 of {r2:.5f}')

Test set have a r2 of 0.94603


#### Some remarks

##### Note1: the two dataset have 66% percentage of repetitive molecules.
##### Note2: The perfomence on chembl and kinect dataset are bad certainly because these dataset contains more molecules.
##### We can train on the chembl dataset with removal of a certain number of outliers to see if their are some difference.